In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [3]:
# read table

dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [4]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

payroll.head(10)

,Fiscal Year,Fiscal Period,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Working Year
0,2016,2016.00,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.00
1,2016,2016.25,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.25
2,2016,2016.50,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.50
3,2016,2016.75,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.75
4,2017,2017.00,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.00
5,2017,2017.25,1172,Assistant State's Attorney,24473.38,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.25
6,2017,2017.50,1172,Assistant State's Attorney,21217.35,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.50
7,2016,2016.00,5049,Residential Model Sr Anal III,17770.86,9500731,f313b1c3-1b1a-4b07-bb75-a8c850a91bac,18.00
8,2016,2016.25,5049,Residential Model Sr Anal III,20800.67,9500731,f313b1c3-1b1a-4b07-bb75-a8c850a91bac,18.25
9,2016,2016.50,5049,Residential Model Sr Anal III,17873.76,9500731,f313b1c3-1b1a-4b07-bb75-a8c850a91bac,18.50


In [17]:
# YANG INI JANGAN DIJALANKAN DULU YA GESSS :'

arr = []

try:
    with open('../cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('../cache'):
        os.mkdir('../cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 1000:
            arr.append(str(_id))
    with open('cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [22]:
# Group Job Code 1172 training dataset <<< PAKAI INI NGGIH

group_1172 = payroll.groupby(['Job Code']).get_group(1172)

_sorted = group_1172.sort_values('Employee Identifier')
fiscal = pd.pivot_table(_sorted, values='Base Pay', index=['Fiscal Period'], columns='Employee Identifier')

_diff = fiscal.diff()

In [23]:
# cleaned display table

_diff.fillna(0, inplace=True)
_diff = _diff.drop(2016.00)

In [47]:
b = _diff.stack()

# a = b.unstack(0)
# a
# a.stack('Fiscal Period')

m = pd.DataFrame(b)
m.stack(level='')

# new = pd.pivot_table(m, columns=['Employee Identifier', 'Fiscal Period', 0])
# fiscal = pd.pivot_table(_sorted, columns=['Employee Identifier', 'Fiscal Period', 'Base Pay'])
# fiscal

Fiscal Period  Employee Identifier                    
2016.25        007ca4de-de2b-44ac-bf71-6eb4cbdc27e2  0    2688.25
               00d20c14-93d4-4560-a10d-3e41c1fafd07  0    2583.05
               00d3e7d2-9397-4a16-a556-80056ebbf456  0    9540.17
               00e833c8-ecbf-4e2c-9b66-9472d1949d79  0    4253.60
               01143446-4dfb-4327-840d-fb8b88fe78c3  0    3717.52
                                                           ...   
2018.25        fef3a6ad-266b-44b5-96f5-2952dc364b38  0    3172.57
               ff131ab6-ce88-43ad-b6c1-64507471dc08  0    2914.84
               ff3dd422-b4e5-4b17-87be-23a558efb62e  0    5348.40
               ff9f0a14-2aa1-4ff5-8ad2-8b4db9a3d719  0    2949.53
               ffa8ef63-a6ed-49c2-8b32-f4bbbba79851  0    2420.68
Length: 8442, dtype: float64

In [ ]:
# query job code indexing

for val in arr:
    new_df = payroll[payroll['Job Code'] == val]
    print(f"DataFrame for job code {val}:")
    display(new_df)